In [7]:
import os
import warnings; warnings.filterwarnings("ignore")
import torch as th
from crewai import (Agent, Task, Crew)
from langchain.llms import Ollama
from crewai_tools import (BaseTool, tool, DirectoryReadTool, FileReadTool, SerperDevTool, WebsiteSearchTool)
from crewai.process import Process
from langchain_openai import ChatOpenAI

In [2]:
device = th.device("cuda" if th.cuda.is_available() else "cpu")
devive_cnt = th.cuda.device_count()
print(f"device = {device}; devive_cnt = {devive_cnt}")
print(f"torch version = {th.__version__}")
print(f"cuda version = {th.version.cuda}")

device = cuda; devive_cnt = 1
torch version = 2.0.1+cu118
cuda version = 11.8


## step-1: 配置默认大模型（OpenAI）

In [3]:
os.environ["OPENAI_API_KEY"] = "Your Key"
os.environ["OPENAI_MODEL_NAME"] = "gpt-3.5-turbo"  # gpt-4
os.environ["SERPER_API_KEY"] = "Your Key"

## step-1: 配置本地大模型（Ollama）

In [3]:
ollama_openhermes = Ollama(model="openhermes")
ollama_solar = Ollama(model="Solar")

## step-2: Agent

### An agent is an autonomous unit programmed to:
- Perform tasks
- Make decisions
- Communicate with other agents

In [4]:
agent = Agent(
    role="Data Analyst",
    goal="Extract actionable insights",
    backstory=(
       "You're a data analyst at a large company."
       "You're responsible for analyzing data and providing insights to the business."
       "You're currently working on a project to analyze the performance of our marketing campaigns."
   ),
    llm=ollama_openhermes,  # Optional, defaulting to "gpt-4" if not specified
    # tools=[my_tool1, my_tool2],  # Optional, defaults to an empty list
    allow_delegation=False,  # Optional, Default is True.
    verbose=True,  # Optional
    # cache=True  # Optional, Default is True.
    # function_calling_llm=my_llm,  # Optional
    # max_iter=25,  # Optional
    # max_rpm=None, # Optional
    # step_callback=my_intermediate_step_callback,  # Optional
)

In [8]:
research_agent = Agent(
    role="Researcher",
    goal="Find and summarize the latest AI news",
    backstory=(
        "You're a researcher at a large company."
        "You're responsible for analyzing data and providing insights to the business."
    ),
    llm=ollama_openhermes,
    allow_delegation=False,
    verbose=True
)

## step-3: Task

### 确保每个任务只设置一种输出类型，以保持清晰的输出

In [ ]:
search_tool = SerperDevTool()

In [11]:
task = Task(
  description="Find and summarize the latest AI news",
  expected_output="A bullet list summary of the top 5 most important AI news",
  agent=research_agent,
  tools=[search_tool],
#   context=[task_2, task_3],
#   async_execution=False,
#   output_json=class,
#   output_file="xxx.json",
#   human_input=False
)

In [14]:
research_ai_task = Task(
    description="Find and summarize the latest AI news",
    expected_output="A bullet list summary of the top 5 most important AI news",
    agent=research_agent,
    async_execution=True,
    tools=[search_tool]
)

In [15]:
research_ops_task = Task(
    description="Find and summarize the latest AI news",
    expected_output="A bullet list summary of the top 5 most important AI news",
    agent=research_agent,
    async_execution=True,
    tools=[search_tool]
)

In [ ]:
write_blog_task = Task(
    description="Write a full blog post about the importance of AI and its latest news",
    expected_output="Full blog post that is 4 paragraphs long",
    agent=writer_agent,
    context=[research_ai_task, research_ops_task]
)

In [ ]:
# The callback function is executed after the task is completed
def callback_function(output):
    # Do something after the task is completed
    # Example: Send an email to the manager
    print(f"""
        Task completed!
        Task: {output.description}
        Output: {output.raw_output}
    """)

research_task = Task(
    description="Find and summarize the latest AI news",
    expected_output="A bullet list summary of the top 5 most important AI news",
    agent=research_agent,
    tools=[search_tool],
    callback=callback_function
)

In [12]:
crew = Crew(
    agents=[research_agent],
    tasks=[research_ai_task, research_ops_task, write_blog_task],
    verbose=2,
    # full_output=True
)

In [ ]:
result = crew.kickoff()
print(result)

In [ ]:
print(
    "Task completed!"
    f"Task: {research_ai_task.output.description}"
    f"Output: {research_ai_task.output.raw_output}"
)

## step-4: Tools

### 工具的主要特点
- Utility: 专为web搜索、数据分析、内容生成 和 代理协作 等任务而设计。
- Integration: 通过无缝地将工具集成到 工作流 中，提高 Agent 的能力。
- Customizability: 提供开发定制工具或利用现有工具的灵活性，以满足 Agent 的特定需求。
- Error Handling: 包含强大的错误处理机制，以确保平稳运行。
- Caching Mechanism: 智能缓存功能，优化性能，减少冗余操作。

### 工具列表
#### Search
- DirectorySearchTool: 用于在目录中搜索的RAG工具，用于在文件系统中导航。
- CodeDocsSearchTool: 为搜索代码文档和相关技术文档而优化的RAG工具。
- GithubSearchTool: 一个用于在GitHub存储库中搜索的RAG工具，用于代码和文档搜索。
- DOCXSearchTool: 一个RAG工具，用于在DOCX文档中搜索，非常适合处理Word文件。
- CSVSearchTool: 一个为在CSV文件中搜索而设计的RAG工具，专门用于处理结构化数据。
- JSONSearchTool: 一个为在JSON文件中搜索而设计的RAG工具，用于结构化数据处理。
- XMLSearchTool: 一个为在XML文件中搜索而设计的RAG工具，适用于结构化数据格式。
- TXTSearchTool: 一个专注于在文本(.txt)文件中搜索的RAG工具，适用于非结构化数据。
- MDXSearchTool: 一个专门用于在Markdown (MDX)文件中搜索的RAG工具，对文档编制很有用。
- PDFSearchTool: 一个用于搜索PDF文档的RAG工具，非常适合处理扫描文档。
- PGSearchTool: 一个优化了在PostgreSQL数据库中搜索的RAG工具，适用于数据库查询。
- RagTool: 一种通用的RAG工具，能够处理各种数据源和类型。
- WebsiteSearchTool: 一个搜索网站内容的RAG工具，优化了网络数据提取。
- YoutubeChannelSearchTool: 一个搜索YouTube频道的RAG工具，用于视频内容分析。
- YoutubeVideoSearchTool: 一个RAG工具，旨在搜索YouTube视频，理想的视频数据提取。
- SerperDevTool: 用于开发目的的专用工具，具有正在开发的特定功能。
#### Scrape
- ScrapeElementFromWebsiteTool: 允许从网站上抓取特定元素，用于目标数据提取。
- ScrapeWebsiteTool: 方便抓取整个网站，理想的全面的数据收集。
#### Read
- DirectoryReadTool: 便于读取和处理目录结构及其内容。
- FileReadTool: 支持从文件中读取和提取数据，支持各种文件格式。

In [ ]:
# Instantiate tools
docs_tool = DirectoryReadTool(directory='./blog-posts')
file_tool = FileReadTool()
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()

In [ ]:
# Create agents
researcher = Agent(
    role='Market Research Analyst',
    goal='Provide up-to-date market analysis of the AI industry',
    backstory='An expert analyst with a keen eye for market trends.',
    tools=[search_tool, web_rag_tool],
    verbose=True
)

writer = Agent(
    role='Content Writer',
    goal='Craft engaging blog posts about the AI industry',
    backstory='A skilled writer with a passion for technology.',
    tools=[docs_tool, file_tool],
    verbose=True
)

In [ ]:
# Define tasks
research = Task(
    description='Research the latest trends in the AI industry and provide a summary.',
    expected_output='A summary of the top 3 trending developments in the AI industry with a unique perspective on their significance.',
    agent=researcher
)

write = Task(
    description='Write an engaging blog post about the AI industry, based on the research analyst’s summary. Draw inspiration from the latest blog posts in the directory.',
    expected_output='A 4-paragraph blog post formatted in markdown with engaging, informative, and accessible content, avoiding complex jargon.',
    agent=writer,
    output_file='blog-posts/new_post.md'  # The final blog post will be saved here
)

In [ ]:
# Assemble a crew
crew = Crew(
    agents=[researcher, writer],
    tasks=[research, write],
    verbose=2
)

In [ ]:
# Execute tasks
crew.kickoff()

In [ ]:
# Creating your own Tools - 1
class MyCustomTool(BaseTool):
    name: str = "Name of my tool"
    description: str = "Clear description for what this tool is useful for, you agent will need this information to use it."

    def _run(self, argument: str) -> str:
        # Implementation goes here
        return "Result from custom tool"

# Creating your own Tools - 2
@tool("Name of my tool")
def my_tool(question: str) -> str:
    """Clear description for what this tool is useful for, you agent will need this information to use it."""
    # Function logic here
    return "Result from your custom tool"

## step-5: Processes

### Process 的实现
- Sequential: 按顺序执行任务，确保任务按顺序完成。
- Hierarchical: 在管理层次结构中组织任务，其中任务根据结构化的命令链进行委派和执行。必须在团队中指定一个管理器语言模型(manager_llm)，以启用分层流程，促进管理器创建和管理任务。
- Consensual Process (Planned): 针对任务执行过程中各代理之间的协作决策，这种流程类型在CrewAI中引入了一种民主的任务管理方法。它计划用于未来的开发，目前还没有在代码库中实现。

In [ ]:
# Example: Creating a crew with a sequential process
crew = Crew(
    agents=my_agents,
    tasks=my_tasks,
    process=Process.sequential,
    manager_llm=None
)

In [ ]:
# Example: Creating a crew with a hierarchical process
# Ensure to provide a manager_llm
crew = Crew(
    agents=my_agents,
    tasks=my_tasks,
    process=Process.hierarchical,
    manager_llm=ChatOpenAI(model="gpt-4")  # hierarchical 必选
)

## step-6: Crew

In [ ]:
# Assemble the crew with a sequential process
my_crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, write_article_task],
    process=Process.sequential,
    memory=True,
    Cache=True,
    full_output=True,
    verbose=True
)

## step-7: Collaboration

## step-8: Memory